Parkinson's Disease is a progressive disorder of the nervous system that affects movement. It develops gradually and cannot be cured, although treatment may help. It requires medical diagnosis and it can occur starting from the age of 6. Thus, this is a preliminary study that tries to identify correlations between Parkinson disease variables for easy identification of the disease.A

In [ ]:
#Krishna Thiyagarajan
#Abhinav Jain
#Linear Reg. Min Project
#ECE-411: Stat Learning
#Prof. Keene


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import statsmodels.api as sm
from sklearn import datasets, linear_model
from sklearn.linear_model import lasso_path
import matplotlib.pyplot as plt

np.set_printoptions(suppress = True, precision=3); #Options for NumPy

df = pd.read_csv('../input/Data.csv', sep = ',', header = 0);  #load CSV file using pandas

df.describe() # Summary of data set indicating mean, std, min etc.

**Analyzing Variables to Find the Dependent Variable**

As it can be seen, the table doesn't tell you what the dependent variable of this data set is. So we must discover it. To do so, we find the correlation matrix and see which column has the highest average correlation constant. This is obviously for demonstration purposes and it cannot be used as a way of determining a "dependent variable". #

**Qualitative Look at Correlation Matrix**

In [ ]:
corrMat = df[::].corr(); 
sns.heatmap(corrMat, vmax=0.8, square = True);

**Quantitative Look at Correlation Matrix**

In [ ]:
print(corrMat);

After some simple calculations, one can see that Jitter(Percent) seems to generally have the highest percent of correlation with all of the other variables. Thus, let us use that as the dependent variable and the rest as the independent variables.

In [ ]:

X = df.values[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]] # First 19 columns of data
y = df.values[:, 19]; # Last col of data
X = np.array(X); 
X = sm.add_constant(X); 
results = sm.OLS(endog=y, exog = X).fit(); 
print(results.summary())

As it can be, only some of the above variables are truly contributing to the regression. In particular, the following do not seem to contribute to the model as much as the other variables because they are less than 2 standard deviations from the mean: x1, x2, x6, x8, x11, x14 and x18. In other words, these variables are accepted by the null hypothesis. Thus, let us see how things change if we ignore these variables. 

In [ ]:
X = df.values[:,[2,3,4,6,8,9,11,12,14,15,16,18]] # First 19 columns of data
y = df.values[:, 19]; # Last col of data
X = np.array(X); 
X = sm.add_constant(X); 
results1 = sm.OLS(endog=y, exog = X).fit(); 

print(results1.summary())

In [ ]:
print("\n\n\nβ =", results1.params) #new beta vector for linear regression

Some of the t-values have changed and the R-squared variable has decreased, but the new model seems to predict the Jitter percentage as well as the old model. Thus, ignoring those certain variables with the low t-scores did not do much damage.

**Let us See how Ridge Regression Works on this Data set**

In [ ]:
X = df.values[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]] # First 19 columns of data
y = df.values[:, 19]; # Last col of data

n_alphas = 200

alphas = np.logspace(-10, -2, n_alphas)
regr = linear_model.Ridge(fit_intercept = False)

coefs = []; 
for a in alphas:
    regr.set_params(alpha = a)
    regr.fit(X,y)
    coefs.append(regr.coef_)
    
ax = plt.gca()
ax.plot(alphas, coefs)
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1])
plt.xlabel('alpha')
plt.ylabel('weights')
plt.title('Ridge coefficients as a function of the regularization')
plt.axis('tight')
plt.show()

In [ ]:
from itertools import cycle

X = df.values[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]] # First 19 columns of data
y = df.values[:, 19]; # Last col of data

eps = 5e-20
alphas_lasso, coefs_lasso, _ = lasso_path(X, y, eps, fit_intercept = False)

plt.figure(1)
ax = plt.gca()
colors = cycle(['b', 'r', 'g', 'c', 'k']);
neg_log_alphas_lasso = -np.log10(alphas_lasso)
for coef_l, c in zip(coefs_lasso, colors):
    l1 = plt.plot(neg_log_alphas_lasso, coef_l, c=c);
   
plt.xlabel('-Log(alpha)')
plt.ylabel('coefficients')
plt.title('Lasso Paths')
plt.axis('tight')




#lassocoefs = []; 
#for a in alphas:
#    regr.set_params(alpha = a)
#    regr.fit(X,y)
#    lassocoefs.append(regr.coef_)

#plt.xlabel('-Log(alpha)');
#plt.ylabel('coefficients');
#plt.title('Lasso Paths');
#plt.axis('tight');

**Comparison of Least Squares, Ridge and Lasso**#

In [ ]:
print("Ridge   ", "Lasso    ", "OLS")

A  = [coefs[0], coefs_lasso.T[::-1][0], results.params[1:]]

print(np.matrix(A).T)